In [8]:
import cv2
import numpy as np
import mediapipe
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
folder = "/Users/gokhanersoz/Desktop/Hepsi/OpenCV/PROJECTS/Resources/Virtual Painter"
myList = os.listdir(folder)
myList.sort()
print(myList)
overlayList = []
for imPath in myList:
    image = cv2.imread(f"{folder}/{imPath}")
    image = cv2.resize(image,(1280,125))
    overlayList.append(image)
print(len(overlayList))

['1.png', '2.png', '3.png', '4.png']
4


In [11]:
import cv2
import mediapipe as mp
import time
import math


class HandDetector:
    
    def __init__(self,mode = False,maxHands = 2, complexity = 1, detectionCon = 0.5,trackCon = 0.5):
        
        self.mode = mode
        self.maxHands = maxHands
        self.complexity = complexity
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        
        self.mpDraw = mp.solutions.drawing_utils
        self.mpHand = mp.solutions.hands
        
        self.Hands = self.mpHand.Hands(self.mode,self.maxHands,self.complexity,
                                       self.detectionCon,self.trackCon)
        
        self.tipIds = [4,8,12,16,20]
        
    def findHands(self,image, draw = True,flipType = True):
        
        imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        self.results = self.Hands.process(imageRGB)
        allHands = []
        h,w,c = image.shape
        
        if self.results.multi_hand_landmarks:
            for handType,handLms in zip(self.results.multi_handedness,self.results.multi_hand_landmarks):
                myHand = {}
                mylmList = []
                xList = []
                yList = []
                
                for id,lm in enumerate(handLms.landmark):
                    px,py,pz = int(lm.x*w),int(lm.y*h),int(lm.z*w)
                    mylmList.append([px,py,pz])
                    xList.append(px)
                    yList.append(py)
                    
                xmin,xmax = min(xList),max(xList) 
                ymin,ymax = min(yList),max(yList)
                
                bboxW,bboxH = xmax-xmin,ymax-ymin
                bbox = xmin,ymin,xmax,ymax
                
                cx,cy = bbox[0] + (bboxW // 2),bbox[1] + (bboxH // 2)
                
                myHand["lmList"] = mylmList
                myHand["bbox"] = bbox
                myHand["center"] = (cx,cy)
                
                if flipType:
                    if handType.classification[0].label == "Right":
                        myHand["type"] = "Left"
                    else:
                        myHand["type"] = "Right"
                        
                else:
                    
                    myHand["type"] = handType.classification[0].label
                    
                allHands.append(myHand)
                
                if draw:
                    
                    self.mpDraw.draw_landmarks(image,handLms,self.mpHand.HAND_CONNECTIONS)
                    
                    cv2.rectangle(image,(bbox[0]-20,bbox[1] - 20),(bbox[2]+20,bbox[3] + 20),
                                 (255,0,255),2)
                    
                    cv2.putText(image,myHand["type"],(bbox[0]-30,bbox[1] -30),cv2.FONT_HERSHEY_PLAIN,
                               2,(255,0,255),2)
                    
        if draw:
            return allHands,image
        
        else:
            return allHands
        
        
    def fingersUp(self,myHand):
            
        myHandType = myHand["type"]
        mylmList = myHand["lmList"]
        fingers = []
        if myHandType == "Right":

            if mylmList[self.tipIds[0]][0] > mylmList[self.tipIds[0] - 1][0]:
                fingers.append(1)
            else:
                fingers.append(0)

        else:

            if mylmList[self.tipIds[0]][0] < mylmList[self.tipIds[0] - 1][0]:
                fingers.append(1)
            else:
                fingers.append(0)

        for id in range(1,5):
            if mylmList[self.tipIds[id]][1] < mylmList[self.tipIds[id] - 2][1]:
                fingers.append(1)
            else:
                fingers.append(0)

        return fingers
        
        
    def findDistance(self,p1,p2,image = None):
            
        x1,y1 = p1
        x2,y2 = p2
        cx,cy =( x1 + x2 ) // 2,(y1 + y2) // 2
        length = math.hypot(x2-x1,y2-y1)
        info = (x1,y1,x2,y2,cx,cy)

        if image is not None:

            cv2.circle(image, (x1,y1),15,(255,0,255),cv2.FILLED)
            cv2.circle(image, (x2,y2),15,(255,0,255),cv2.FILLED)
            cv2.line(image, (x1,y1),(x2,y2),(255,0,255),3)
            cv2.circle(image, (cx,cy),15,(255,0,255),cv2.FILLED)
            return length,info,image

        return length,info

In [59]:
pTime = 0
def fps():
    global pTime
    
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    return fps


hand_detector = HandDetector(maxHands=1)

cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

while True:
    
    
    conn,frame = cap.read()
    
    allHands,frame = hand_detector.findHands(frame,draw = True,flipType = True)
    
    if allHands:
        
        hand1 = allHands[0]
        lmList = hand1["lmList"]
        center = hand1["center"]
        bbox = hand1["bbox"]
        type1 = hand1["type"]
        
        fingers = hand_detector.fingersUp(hand1)
        #time.sleep(1)
        #print(fingers)
        length1,info1,frame = hand_detector.findDistance(lmList[4][:-1],lmList[8][:-1],frame)
    
    fps_ = fps()
    cv2.putText(frame,f"FPS : {int(fps_)}",(20,70),cv2.FONT_HERSHEY_PLAIN,3,(255,0,255),3)
    cv2.imshow("Frame",frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
        
cap.release()
cv2.destroyAllWindows()

In [1]:
import cv2
import mediapipe as mp
import time
import math

class HandDetector:
    
    def __init__(self,mode = False,maxHands = 2, complexity = 1, detectionCon = 0.5,trackCon = 0.5):
        
        self.mode = mode
        self.maxHands = maxHands
        self.complexity = complexity
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        
        self.mpDraw = mp.solutions.drawing_utils
        self.mpHand = mp.solutions.hands
        
        self.Hands = self.mpHand.Hands(self.mode,self.maxHands,self.complexity,
                                       self.detectionCon,self.trackCon)
        
        self.tipIds = [4,8,12,16,20]
        
    def findHands(self,image, draw = True,flipType = True):
        
        imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        self.results = self.Hands.process(imageRGB)
        allHands = []
        h,w,c = image.shape
        
        if self.results.multi_hand_landmarks:
            for handType,handLms in zip(self.results.multi_handedness,self.results.multi_hand_landmarks):
                myHand = {}
                mylmList = []
                xList = []
                yList = []
                
                for id,lm in enumerate(handLms.landmark):
                    px,py,pz = int(lm.x*w),int(lm.y*h),int(lm.z*w)
                    mylmList.append([px,py,pz])
                    xList.append(px)
                    yList.append(py)
                    
                xmin,xmax = min(xList),max(xList) 
                ymin,ymax = min(yList),max(yList)
                
                bboxW,bboxH = xmax-xmin,ymax-ymin
                bbox = xmin,ymin,xmax,ymax
                
                cx,cy = bbox[0] + (bboxW // 2),bbox[1] + (bboxH // 2)
                
                myHand["lmList"] = mylmList
                myHand["bbox"] = bbox
                myHand["center"] = (cx,cy)
                
                if flipType:
                    if handType.classification[0].label == "Right":
                        myHand["type"] = "Left"
                    else:
                        myHand["type"] = "Right"
                        
                else:
                    
                    myHand["type"] = handType.classification[0].label
                    
                allHands.append(myHand)
                
                if draw:
                    
                    self.mpDraw.draw_landmarks(image,handLms,self.mpHand.HAND_CONNECTIONS)
                    
                    cv2.rectangle(image,(bbox[0]-20,bbox[1] - 20),(bbox[2]+20,bbox[3] + 20),
                                 (255,0,255),2)
                    
                    cv2.putText(image,myHand["type"],(bbox[0]-30,bbox[1] -30),cv2.FONT_HERSHEY_PLAIN,
                               2,(255,0,255),2)
                    
        if draw:
            return allHands,image
        
        else:
            return allHands
        
        
    def fingersUp(self,myHand,flipType=True):
            
        myHandType = myHand["type"]
        mylmList = myHand["lmList"]
        fingers = []
        if flipType :
            if myHandType == "Right":

                if mylmList[self.tipIds[0]][0] > mylmList[self.tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)

            else:

                if mylmList[self.tipIds[0]][0] < mylmList[self.tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)
                    
        else:
            if myHandType == "Right":

                if mylmList[self.tipIds[0]][0] < mylmList[self.tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)

            else:

                if mylmList[self.tipIds[0]][0] > mylmList[self.tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)
                    
            

        for id in range(1,5):
            if mylmList[self.tipIds[id]][1] < mylmList[self.tipIds[id] - 2][1]:
                fingers.append(1)
            else:
                fingers.append(0)

        return fingers
        
        
    def findDistance(self,p1,p2,image = None):
            
        x1,y1 = p1
        x2,y2 = p2
        cx,cy =( x1 + x2 ) // 2,(y1 + y2) // 2
        length = math.hypot(x2-x1,y2-y1)
        info = (x1,y1,x2,y2,cx,cy)

        if image is not None:

            cv2.circle(image, (x1,y1),15,(255,0,255),cv2.FILLED)
            cv2.circle(image, (x2,y2),15,(255,0,255),cv2.FILLED)
            cv2.line(image, (x1,y1),(x2,y2),(255,0,255),3)
            cv2.circle(image, (cx,cy),15,(255,0,255),cv2.FILLED)
            return length,info,image

        return length,info

In [10]:
pTime = 0
def fps():
    global pTime
    
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    return fps

header = overlayList[0]
brushThickness = 15
eraserThickness = 70
drawColor = (255,105,180)
xp,yp = 0,0
imgCanvas = np.zeros((720,1280,3),np.uint8)

video_writer = cv2.VideoWriter("virtual.mp4",cv2.VideoWriter_fourcc(*"XVID"),30,(640,480))

cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)

hand_detector = HandDetector(maxHands=1)

while True:
    
    # 1. Import image
    conn,frame = cap.read()
    frame = cv2.flip(frame,1)
    
    # 2.Find Hand Landmarks
    allHands,frame = hand_detector.findHands(frame, draw = True, flipType = False)
    
    if allHands:
        
        hands1 = allHands[0]
        lmList1 = hands1["lmList"]
        bbox1 = hands1["bbox"]
        center1 = hands1["center"]
        type1 = hands1["type"]
        
        x1,y1 = lmList1[8][:-1]
        x2,y2 = lmList1[12][:-1]
        
        #length1,info1,frame = hand_detector.findDistance(lmList1[12][:-1],lmList1[8][:-1],frame)
        
    # 3. Check which fingers are up
        fingers = hand_detector.fingersUp(hands1,flipType = False)
        #print(fingers)
        
    # 4. If Selection Mode - Two Fingers are up
        if fingers[1] and fingers[2]:
            
            xp,yp = 0, 0
            
            cv2.rectangle(frame,(x1,y1-25),(x2,y2+25),drawColor,cv2.FILLED)
            cv2.putText(frame,"Mode : ",(10,250),cv2.FONT_HERSHEY_PLAIN,2,(255,0,255),2)
            cv2.putText(frame,"Selection Mode ",(150,250),cv2.FONT_HERSHEY_PLAIN,2,(255,0,255),2)
            
            # Checking For The Click
            if y1 < 125:
                if 250 < x1 < 450:
                    header = overlayList[0]
                    drawColor = (255,105,180)
                
                if 550 < x1 < 750:
                    header = overlayList[1]
                    drawColor = (0,0,255)
                
                if 800 < x1 < 950:
                    header = overlayList[2]
                    drawColor = (0,255,0)
                
                if 1050 < x1 < 1200:
                    header = overlayList[3]
                    drawColor = (0,0,0)
                      
    # 5. If Drawing Mode - Index Finger is up
        if fingers[1] and fingers[2] == False:
            cv2.circle(frame,(x1,y1),15,drawColor,cv2.FILLED)
            cv2.putText(frame,f"Mode : ",(10,250),cv2.FONT_HERSHEY_PLAIN,2,(255,0,255),2)
            cv2.putText(frame,"Drawing Mode ",(150,250),cv2.FONT_HERSHEY_PLAIN,2,(255,0,255),2)
            
            if xp == 0 and yp == 0:
                xp,yp = x1,y1
            
            if drawColor == (0,0,0):
                cv2.line(frame,(xp,yp),(x1,y1),drawColor,eraserThickness)
                # Arka taraf siyah çizim seçilen renk !!!
                # Silmek içinde siyah rengi kullandık 
                # Çünkü siyah renk renklileri absorde edicek !!!
                cv2.line(imgCanvas,(xp,yp),(x1,y1),drawColor,eraserThickness)
            
            else:
                
                cv2.line(frame,(xp,yp),(x1,y1),drawColor,brushThickness) 
                # Arka taraf siyah çizim seçilen renk !!!
                # Silmek içinde siyah rengi kullandık 
                # Çünkü siyah renk renklileri absorde edicek !!!
                cv2.line(imgCanvas,(xp,yp),(x1,y1),drawColor,brushThickness)
                
            xp,yp = x1,y1
            
    imgGray = cv2.cvtColor(imgCanvas,cv2.COLOR_BGR2GRAY)  
    
    # Arka taraf beyaz çizimler siyah !!!
    _,imgInv = cv2.threshold(imgGray,50,255,cv2.THRESH_BINARY_INV)
            
    imgInv = cv2.cvtColor(imgInv,cv2.COLOR_GRAY2BGR)
    
    # Arka taraf ekran görüntüsü çizimler siyah !!!
    frame_ = cv2.bitwise_and(frame,imgInv)
    
    # Arka Taraf ekran görüntüsü çizimler renkli !!! 
    # Çünkü imgCanvas da arka taraf siyah çizimler renkli !!!
    # Siyah çizimler imgCanvas ile toplanınca renkli oluyor !!!
    frame = cv2.bitwise_or(frame_,imgCanvas)

    # Setting The Header Image
    frame[0:125,0:1280] = header
    fps_ = fps()
    cv2.putText(frame,f"FPS : {int(fps_)}",(10,170),cv2.FONT_HERSHEY_PLAIN,2,(255,0,255),2)
    
    hstack1 = np.hstack([imgCanvas,imgInv])
    hstack2 = np.hstack([frame_,frame])
    vstack1 = np.vstack([hstack1,hstack2])
    
    #frame = cv2.addWeighted(frame,0.5,imgCanvas,0.5,0)
    
    video_writer.write(frame)
    cv2.imshow("Frame",frame)

    #cv2.imshow("Canvas",imgCanvas)
    #cv2.imshow("Gray",imgGray)
    #cv2.imshow("Triple",vstack1)
    
    if cv2.waitKey(1) & 0xff == ord("q"):
        break
video_writer.release()       
cap.release()
cv2.destroyAllWindows()